# MEWpy Community Modeling

Author: Vitor Pereira, inspired on the work by Daniel Machado. 

License: [CC BY-SA 4.0](http://creativecommons.org/licenses/by-sa/4.0/)

-------

In this tutorial:

- You will learn how to perform flux balance analysis of microbial communities
using a model of the [central carbon metabolism of *E. coli*](https://journals.asm.org/doi/10.1128/ecosalplus.10.2.1).


## Install requirements 
To run this notebook we firstly need to install the required packages

Verify the instalation

In [1]:
import mewpy
mewpy.info()

MEWpy version: 0.1.28
Author: BiSBII CEB University of Minho
Contact: vpereira@ceb.uminho.pt 

Available LP solvers: gurobi glpk
Default LP solver: gurobi 

Available ODE solvers: scipy
Default ODE solver: scipy 

Optimization Problems: AbstractKOProblem AbstractOUProblem CommunityKOProblem ETFLGKOProblem ETFLGOUProblem GKOProblem GOUProblem GeckoKOProblem GeckoOUProblem KcatOptProblem KineticKOProblem KineticOUProblem MediumProblem OptORFProblem OptRamProblem RKOProblem ROUProblem 

Available EA engines: inspyred jmetal
Default EA engine: jmetal
Available EAs: GA NSGAII NSGAIII SA SPEA2 



In [2]:
from mewpy.solvers import set_default_solver,get_default_solver
set_default_solver('gurobi')

In [3]:
get_default_solver()

'gurobi'

IMPORTANT: The notebooks require a MEWpy version >= 0.1.26

### Run in Google colab

If you are running this notebook in Colab, you need to perform the following steps, otherwise skip.

## Setting up a community

We will create a synthetic microbial consortium with two *E. coli* mutants growing in minimal medium. In one of the mutants we will knockout the glucose transporter and in the other we will knockout the ammonium transporter.

In [4]:
from cobra.io import read_sbml_model

bt = read_sbml_model('../models/non-ec/ncbi_refseq/vpi5482.xml')
bu = read_sbml_model('../models/non-ec/ncbi_refseq/atcc8492.xml')
ec = read_sbml_model('../models/non-ec/ncbi_refseq/ed1a.xml')
cc = read_sbml_model('../models/non-ec/ncbi_refseq/atcc27758.xml')
ri = read_sbml_model('../models/non-ec/ncbi_refseq/l182.xml')
sp = read_sbml_model('../models/non-ec/ncbi_refseq/atcc15912.xml')

Set parameter Username


## Comparing models

Community models require that metabolites have the same identifiers accros all models. MEWpy offers some functions tho that end, computing the metabolites, reactions and uptakes overlaps between a list models.

In [5]:
from mewpy.cobra.com import *

mets, rxns, over = jaccard_similarity_matrices([bt,bu,ec,cc,ri,sp])

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the mos

In [6]:
mets

,vpi5482,atcc8492,ed1a,atcc27758,l182,atcc15912
vpi5482,1.000000,0.859259,0.540873,0.602206,0.614317,0.538695
atcc8492,0.859259,1.000000,0.536517,0.594395,0.632911,0.536287
ed1a,0.540873,0.536517,1.000000,0.484069,0.527873,0.445727
atcc27758,0.602206,0.594395,0.484069,1.000000,0.698553,0.605154
l182,0.614317,0.632911,0.527873,0.698553,1.000000,0.612161
atcc15912,0.538695,0.536287,0.445727,0.605154,0.612161,1.000000


In [7]:
rxns

,vpi5482,atcc8492,ed1a,atcc27758,l182,atcc15912
vpi5482,1.000000,0.770921,0.378037,0.420721,0.460503,0.355149
atcc8492,0.770921,1.000000,0.371447,0.421101,0.491604,0.359382
ed1a,0.378037,0.371447,1.000000,0.303748,0.341471,0.284207
atcc27758,0.420721,0.421101,0.303748,1.000000,0.528866,0.403887
l182,0.460503,0.491604,0.341471,0.528866,1.000000,0.419867
atcc15912,0.355149,0.359382,0.284207,0.403887,0.419867,1.000000


In [8]:
over

,vpi5482,atcc8492,ed1a,atcc27758,l182,atcc15912
vpi5482,1.000000,0.731343,0.387097,0.474308,0.476190,0.417004
atcc8492,0.731343,1.000000,0.362398,0.439516,0.489540,0.391667
ed1a,0.387097,0.362398,1.000000,0.360104,0.412399,0.371585
atcc27758,0.474308,0.439516,0.360104,1.000000,0.530120,0.534188
l182,0.476190,0.489540,0.412399,0.530120,1.000000,0.523404
atcc15912,0.417004,0.391667,0.371585,0.534188,0.523404,1.000000


## Building communities

**MEWpy** has some basic functionality for working with microbial communities, one is the `CommunityModel` class to create microbial communities from a list of models of individual species: 

In [9]:
from mewpy.model import CommunityModel
community = CommunityModel([bt,bu,ec,cc,ri,sp],flavor='cobra')

In [10]:
sim = community.merged_model

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the mos

In [11]:
print(len(sim.reactions))

9328


This community model ignores the environmental conditions that were specified in the original models (since these could be very different). 

To make our life easier, we will extract the nutrient composition specified in the wild-type model to use later.

In [12]:
from mewpy.simulation import Environment
M9 = Environment.from_model(bt)
M9

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


EX_12ppd__S_e	-1000.0	1000.0
EX_14glucan_e	-1000.0	1000.0
EX_2ameph_e	-1000.0	1000.0
EX_2m35mdntha_e	-1000.0	1000.0
EX_2pglyc_e	-1000.0	1000.0
EX_35dnta_e	-1000.0	1000.0
EX_4abut_e	-1000.0	1000.0
EX_5drib_e	-1000.0	1000.0
EX_5mcsn_e	-1000.0	1000.0
EX_LalaDgluMdap_e	-1000.0	1000.0
EX_LalaLglu_e	-1000.0	1000.0
EX_Larab_e	-1000.0	1000.0
EX_R3hdec4e_e	-1000.0	1000.0
EX_R_3hdcaa_e	-1000.0	1000.0
EX_R_3hphxa_e	-1000.0	1000.0
EX_R_3hppta_e	-1000.0	1000.0
EX_R_3hpt_e	-1000.0	1000.0
EX_abg4_e	-1000.0	1000.0
EX_acald_e	-1000.0	1000.0
EX_acmana_e	-1000.0	1000.0
EX_actn__R_e	-1000.0	1000.0
EX_ad_e	-1000.0	1000.0
EX_ade_e	-1000.0	1000.0
EX_adn_e	-1000.0	1000.0
EX_ala__D_e	-1000.0	1000.0
EX_ala__L_e	-1000.0	1000.0
EX_alaala_e	-1000.0	1000.0
EX_alltn_e	-1000.0	1000.0
EX_anhgm_e	-1000.0	1000.0
EX_arab__L_e	-1000.0	1000.0
EX_arg__L_e	-1000.0	1000.0
EX_asn__L_e	-1000.0	1000.0
EX_aso3_e	-1000.0	1000.0
EX_aso4_e	-1000.0	1000.0
EX_asp__L_e	-1000.0	1000.0
EX_btn_e	-1000.0	1000.0
EX_bz_e	-1000.0	1000.0
EX_ca

## Simulation using FBA

A very simple way to simulate a microbial community is to merge the individual models into a single model that mimics a "super organism", where each microbe lives inside its own compartment, and run a (conventional) FBA simulation for this *super organism*.

In [13]:
solution = sim.simulate(constraints=M9)

print(solution)
solution.find('EX')

objective: 245.98446119143432
Status: OPTIMAL
Method:SimulationMethod.FBA


,Flux rate
Reaction ID,
DEXTRINt2_atcc27758,462.318800
HEXTT_l182,0.005246
EX_2pglyc_e,-25.349066
EX_5mcsn_e,-0.005940
EX_LalaDgluMdap_e,996.764163
...,...
EX_citr__L_e,-2.855694
EX_dextrin_e,-462.318800
EX_uaccg_e,-6.905670


We can see that the model predicts a growth rate (total biomass per hour) similar to the wild-type, with an efficient consumption of glucose and ammonia that results in respiratory metabolism.

But what is each organism doing, and are both organisms actually growing at the same rate?

Let's print the biomass flux for each organism:

In [14]:
solution.find('Growth', sort=True,show_nulls=True)

,Flux rate
Reaction ID,
Growth_atcc27758,9.654401
Growth_ed1a,32.358368
Growth_atcc8492,42.874066
Growth_vpi5482,49.230500
Growth_l182,52.464832
Growth_atcc15912,59.402295


and all non null fluxes by organism:

In [15]:
sim.find_metabolites()

,name,compartment,formula
id,,,
community_biomass,Total community biomass,e,None
10fthf_c_vpi5482,10-Formyltetrahydrofolate,C_c_vpi5482,C20H21N7O7
12dgr120_c_vpi5482,"1,2-Diacyl-sn-glycerol (didodecanoyl, n-C12:0)",C_c_vpi5482,C27H52O5
12dgr120_p_vpi5482,"1,2-Diacyl-sn-glycerol (didodecanoyl, n-C12:0)",C_p_vpi5482,C27H52O5
12dgr140_c_vpi5482,"1,2-Diacyl-sn-glycerol (ditetradecanoyl, n-C14:0)",C_c_vpi5482,C31H60O5
...,...,...,...
prbamp_c_atcc15912,1-(5-Phosphoribosyl)-AMP,C_c_atcc15912,C15H19N5O14P2
prbatp_c_atcc15912,1-(5-Phosphoribosyl)-ATP,C_c_atcc15912,C15H19N5O20P4
prfp_c_atcc15912,1-(5-Phosphoribosyl)-5-[(5-phosphoribosylamino...,C_c_atcc15912,C15H21N5O15P2


In [16]:
solution.find('vpi5482')

,Flux rate
Reaction ID,
4HTHRA_vpi5482,-106.054588
4HTHRK_vpi5482,106.054588
ACALDt_vpi5482,-113.492184
ACKr_vpi5482,4.508529
ACLS_vpi5482,354.326935
...,...
VALTA_vpi5482,-20.832477
r0280_vpi5482,10.589284
r0963_vpi5482,0.032935


Actually it seems that only one of the organisms is growing while the other has an active metabolism (it exchanges metabolites with the environment and with the other organism) performing the role of a bioconverter, but none of the flux is used for growth. 

> Do you think this would be a stable consortium ?

## Community Simulation with SteadyCom

**SteadyCom** by [Chan, et al (2017)](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1005539) is a recent community simulation method that takes into account the fact that to reach a stable composition the organisms need to grow at the same *specific growth rate* (1/h), which means that the *absolute growth rate* (gDW/h) of each organism is proportional to its *abundance* at steady-state (gDW).

Let's simulate the same community using SteadyCom:

In [17]:
from mewpy.cobra.com import SteadyCom

solution = SteadyCom(community,constraints = M9)

In this case the solution object shows the overall community growth rate and the relative abundance of each species:

In [18]:
solution

Community growth: 80.7626953125
vpi5482	0.0
atcc8492	0.0
ed1a	1.0
atcc27758	0.0
l182	0.0
atcc15912	0.0

The `solution` object for community simulations implements a few additional features, such as enumerating all the cross-feeding interactions:

In [19]:
solution.cross_feeding(as_df=True).dropna().sort_values('rate', ascending=False)

,donor,receiver,compound,rate


We can plot the fluxes of each mutant in a map to help with interpretation of the results:

In [21]:
from mewpy.visualization.escher import build_escher

build_escher(fluxes=solution.internal['vpi5482'])

Builder(reaction_data={'12DGR120tipp': 0.0, '12DGR140tipp': 0.0, '12DGR141tipp': 0.0, '12DGR160tipp': 0.0, '12…

In [22]:
build_escher(fluxes=solution.internal['atcc8492'])

Builder(reaction_data={'12DGR120tipp': 0.0, '12DGR140tipp': 0.0, '12DGR141tipp': 0.0, '12DGR160tipp': 0.0, '12…

In [23]:
build_escher(fluxes=solution.internal['ed1a'])

Builder(reaction_data={'12DGR140tipp': 0.0, '12DGR160tipp': 0.0, '12DGR180tipp': 0.0, '12DGR181tipp': 0.0, '12…

In [24]:
build_escher(fluxes=solution.internal['atcc27758'])

Builder(reaction_data={'12DGR120tipp': 0.0, '12DGR140tipp': 0.0, '12DGR141tipp': 0.0, '12DGR160tipp': 0.0, '12…

In [25]:
build_escher(fluxes=solution.internal['atcc15912'])

Builder(reaction_data={'12DGR120tipp': 0.0, '12DGR140tipp': 0.0, '12DGR141tipp': 0.0, '12DGR160tipp': 0.0, '12…

In [26]:
build_escher(fluxes=solution.internal['l182'])

Builder(reaction_data={'12DGR120tipp': 0.0, '12DGR140tipp': 0.0, '12DGR141tipp': 0.0, '12DGR160tipp': 0.0, '12…

## Explore alternative solutions

Unfortunately, one limitation of **SteadyCom**, which is exemplified by [Chan, et al (2017)](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1005539) in Figure 3 (reproduced below), is the variability in the solution space when the community is not growing at the maximum (theoretical) growth rate.

> Would you expect a synthetic community to grow at its maximum growth rate?

**MEWpy** implements a variability analysis function for the SteadyCom solution space, let's see what happens if the community is growing at 90% of the theoretical maximum:

In [27]:
from mewpy.cobra.com import SteadyComVA
l_va = np.linspace(0.1,1.0,num=10)

for va in l_va:
    va = round(va, 1)
    variability = SteadyComVA(community, obj_frac=va, constraints=M9)
    print(f'Strain\tMin\tMax\tVariability - {va}')
    for strain, (lower, upper) in variability.items():
        print(f'{strain}\t{lower:.1%}\t{upper:.1%}')

Strain	Min	Max	Variability - 0.1
vpi5482	0.0%	100.0%
atcc8492	0.0%	100.0%
ed1a	0.0%	100.0%
atcc27758	0.0%	100.0%
l182	0.0%	100.0%
atcc15912	0.0%	100.0%
Strain	Min	Max	Variability - 0.2
vpi5482	0.0%	100.0%
atcc8492	0.0%	100.0%
ed1a	0.0%	100.0%
atcc27758	0.0%	100.0%
l182	0.0%	100.0%
atcc15912	0.0%	100.0%
Strain	Min	Max	Variability - 0.3
vpi5482	0.0%	100.0%
atcc8492	0.0%	100.0%
ed1a	0.0%	100.0%
atcc27758	0.0%	100.0%
l182	0.0%	100.0%
atcc15912	0.0%	100.0%
Strain	Min	Max	Variability - 0.4
vpi5482	0.0%	100.0%
atcc8492	0.0%	100.0%
ed1a	0.0%	100.0%
atcc27758	0.0%	95.6%
l182	0.0%	100.0%
atcc15912	0.0%	100.0%
Strain	Min	Max	Variability - 0.5
vpi5482	0.0%	100.0%
atcc8492	0.0%	100.0%
ed1a	0.0%	100.0%
atcc27758	0.0%	76.8%
l182	0.0%	100.0%
atcc15912	0.0%	100.0%
Strain	Min	Max	Variability - 0.6
vpi5482	0.0%	100.0%
atcc8492	0.0%	-0.0%
ed1a	0.0%	100.0%
atcc27758	0.0%	-0.0%
l182	0.0%	100.0%
atcc15912	0.0%	100.0%
Strain	Min	Max	Variability - 0.7
vpi5482	0.0%	100.0%
atcc8492	0.0%	-0.0%
ed1a	0.0%	100.0%
at

As you can see, there is a really large variability in this solution space. This means that we know in theory the two mutants **can** cooperate and survive in minimal media, but there is still a lot of uncertainty with regard to **how** they will achieve a stable consortium.

> How do you think we can reduce this uncertainty?

Firstly, lets set the environment conditions:

In [28]:
sim.set_environmental_conditions(M9)

We may now impose constraints on each organism growth, such as stating that each organism need to grow at least 0.1/h

In [29]:
constraints={community.organisms_biomass['atcc8492']:(0.1,1000), 
             community.organisms_biomass['vpi5482']:(0.1,1000), 
             community.organisms_biomass['ed1a']:(0.1,1000),
             community.organisms_biomass['atcc27758']:(0.1,1000),
             community.organisms_biomass['l182']:(0.1,1000),
             community.organisms_biomass['atcc15912']:(0.1,1000)}
solution = sim.simulate(constraints=constraints)
solution

objective: 245.98446119143424
Status: OPTIMAL
Method:SimulationMethod.FBA

In [30]:
solution.find('Growth')

,Flux rate
Reaction ID,
Growth_vpi5482,49.230500
Growth_atcc8492,42.874066
Growth_ed1a,32.358368
Growth_atcc27758,9.654401
Growth_l182,52.464832
Growth_atcc15912,59.402295


Alternatively, we might choose to impose relative growth rates for each of the organisms:

In [ ]:
community2 = CommunityModel([bt,bu],
                           add_compartments=True,
                           merge_biomasses=True,
                           flavor='cobra')

In [ ]:
sim2 = community2.get_community_model()
sim2.set_environmental_conditions(M9)

In [ ]:
solution = sim2.simulate()
print(solution)
solution.find('BIOMASS')

In [ ]:
sim2.find(community2.biomass)

The relative abundance (relative growth rates) are by default equal. We may though change these ratios:  

In [ ]:
community2.set_abundance({'glc_ko':1,'nh4_ko':2.5})
sim2.simulate().find('BIOMASS')

## SMETANA

**SMETANA** implements several algorithms to analyse cross-feeding interactions in microbial communities. These have been describe in [Zelezniak et al, PNAS (2015)](https://www.pnas.org/doi/abs/10.1073/pnas.1421834112). Please read the paper for a more detailed explanation.

SCS (species coupling score): measures the dependency of one species in the presence of the others to survive

In [42]:
SCC = sc_score(community)

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the mos

In [43]:
pd.DataFrame.from_dict(SCC)

,vpi5482,atcc8492,ed1a,atcc27758,l182,atcc15912
atcc27758,0.0,0.0,0.0,NaN,0.0,0.0
l182,0.0,0.0,0.0,0.0,NaN,0.0
ed1a,0.0,0.0,NaN,0.0,0.0,0.0
atcc8492,0.0,NaN,0.0,0.0,0.0,0.0
atcc15912,0.0,0.0,0.0,0.0,0.0,NaN
vpi5482,NaN,0.0,0.0,0.0,0.0,0.0


MUS (metabolite uptake score): measures how frequently a species needs to uptake a metabolite to survive

In [32]:
MUS = mu_score(community)
pd.DataFrame.from_dict(MUS)

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the mos

,vpi5482,atcc8492,ed1a,atcc27758,l182,atcc15912
12ppd__S_e,0.00,NaN,0.00,0.0,NaN,NaN
14glucan_e,0.06,NaN,0.37,NaN,NaN,0.0
2ameph_e,0.34,0.17,0.01,NaN,NaN,NaN
2m35mdntha_e,0.00,0.00,0.00,NaN,NaN,NaN
2pglyc_e,0.00,0.06,0.00,0.0,0.0,0.0
...,...,...,...,...,...,...
2hxmp_e,NaN,NaN,NaN,NaN,NaN,0.0
3mba_e,NaN,NaN,NaN,NaN,NaN,0.0
acglu_e,NaN,NaN,NaN,NaN,NaN,0.0
cell4_e,NaN,NaN,NaN,NaN,NaN,0.0


In [34]:
MUS.vpi5482

{'12ppd__S_e': 0.0,
 '14glucan_e': 0.06,
 '2ameph_e': 0.34,
 '2m35mdntha_e': 0.0,
 '2pglyc_e': 0.0,
 '35dnta_e': 0.0,
 '4abut_e': 0.0,
 '5drib_e': 0.0,
 '5mcsn_e': 0.01,
 'LalaDgluMdap_e': 0.0,
 'LalaLglu_e': 0.01,
 'Larab_e': 0.0,
 'R3hdec4e_e': 0.0,
 'R_3hdcaa_e': 0.0,
 'R_3hphxa_e': 0.0,
 'R_3hppta_e': 0.0,
 'R_3hpt_e': 0.0,
 'abg4_e': 0.0,
 'acald_e': 0.0,
 'acmana_e': 0.0,
 'actn__R_e': 0.0,
 'ad_e': 0.01,
 'ade_e': 0.0,
 'adn_e': 0.0,
 'ala__D_e': 0.0,
 'ala__L_e': 0.01,
 'alaala_e': 0.0,
 'alltn_e': 0.0,
 'anhgm_e': 0.01,
 'arab__L_e': 0.0,
 'arg__L_e': 0.0,
 'asn__L_e': 0.0,
 'aso3_e': 0.0,
 'aso4_e': 0.0,
 'asp__L_e': 0.02,
 'btn_e': 0.0,
 'bz_e': 0.0,
 'ca2_e': 1.0,
 'cellb_e': 0.0,
 'cgly_e': 0.37,
 'chol_e': 0.0,
 'chols_e': 0.47,
 'cit_e': 0.0,
 'cl_e': 1.0,
 'co2_e': 0.0,
 'coa_e': 0.34,
 'cobalt2_e': 1.0,
 'crn_e': 0.0,
 'csn_e': 0.01,
 'cu2_e': 1.0,
 'cys__L_e': 0.21,
 'cytd_e': 0.0,
 'd23hb_e': 0.0,
 'dad_2_e': 0.0,
 'dca_e': 0.0,
 'dcyt_e': 0.0,
 'dgsn_e': 0.0,
 'din_

In [35]:
MUS.atcc8492

{'2ameph_e': 0.17,
 '2m35mdntha_e': 0.0,
 '2pglyc_e': 0.06,
 '35dnta_e': 0.0,
 '3amp_e': 0.06,
 '3cmp_e': 0.58,
 '4abut_e': 0.0,
 '4ahmmp_e': 0.0,
 '4hpro_LT_e': 0.0,
 '5mcsn_e': 0.0,
 '5mdru1p_e': 0.0,
 'LalaDgluMdapDala_e': 0.0,
 'LalaLglu_e': 0.0,
 'Larab_e': 0.0,
 'R_3hphxa_e': 0.0,
 'R_3hppta_e': 0.0,
 'R_3hpt_e': 0.0,
 'abg4_e': 0.0,
 'acald_e': 0.0,
 'acnam_e': 0.0,
 'actn__R_e': 0.0,
 'adn_e': 0.0,
 'agm_e': 0.0,
 'ala__D_e': 0.0,
 'ala__L_e': 0.0,
 'alltn_e': 0.0,
 'anhgm_e': 0.0,
 'arab__L_e': 0.0,
 'arg__L_e': 0.0,
 'asn__L_e': 0.0,
 'asp__L_e': 0.0,
 'btn_e': 0.0,
 'bz_e': 0.0,
 'ca2_e': 1.0,
 'cellb_e': 0.0,
 'cgly_e': 0.0,
 'chol_e': 0.0,
 'chols_e': 0.51,
 'cit_e': 0.0,
 'cl_e': 1.0,
 'co2_e': 0.0,
 'coa_e': 0.0,
 'cobalt2_e': 1.0,
 'crn_e': 0.0,
 'cu2_e': 1.0,
 'cys__L_e': 0.0,
 'cytd_e': 0.0,
 'd23hb_e': 0.0,
 'dad_2_e': 0.0,
 'dca_e': 0.0,
 'dcyt_e': 0.0,
 'dgsn_e': 0.0,
 'dhptd_e': 0.0,
 'din_e': 0.0,
 'drib_e': 0.0,
 'duri_e': 0.0,
 'enter_e': 0.0,
 'fald_e': 0.0,
 

MPS (metabolite production score): measures the ability of a species to produce a metabolite

In [36]:
MPS = mp_score(community,environment=M9)
MPS

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the mos

                                                       value
attribute                                                   
vpi5482    {'35dnta_e': 1, 'acald_e': 1, 'arab__L_e': 1, ...
atcc8492   {'acald_e': 1, 'co2_e': 1, 'd23hb_e': 1, 'fald...
ed1a       {'2m35mdntha_e': 1, '4abut_e': 1, 'LalaDgluMda...
atcc27758  {'2mpa_e': 1, 'ac_e': 1, 'co2_e': 1, 'dha_e': ...
l182       {'2mba_e': 1, '2mpa_e': 1, 'acald_e': 1, 'co2_...
atcc15912  {'2mba_e': 1, '3mba_e': 1, 'acald_e': 1, 'aso3...

In [39]:
pd.DataFrame.from_dict(MPS)

,vpi5482,atcc8492,ed1a,atcc27758,l182,atcc15912
35dnta_e,1.0,1.0,1.0,NaN,NaN,NaN
acald_e,1.0,1.0,1.0,1.0,1.0,1.0
arab__L_e,1.0,1.0,0.0,0.0,1.0,0.0
co2_e,1.0,1.0,1.0,1.0,1.0,1.0
d23hb_e,1.0,1.0,NaN,1.0,NaN,NaN
...,...,...,...,...,...,...
3mba_e,NaN,NaN,NaN,NaN,NaN,1.0
cell4_e,NaN,NaN,NaN,NaN,NaN,1.0
2hxmp_e,NaN,NaN,NaN,NaN,NaN,0.0
acglu_e,NaN,NaN,NaN,NaN,NaN,0.0


MRO (metabolic resource overlap): calculates how much the species compete for the same metabolites.

In [40]:
score, MRO = mro_score(community,environment=M9)
print(score)
MRO

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


None


/home/mmartins/anaconda3/envs/tmcom/lib/python3.9/site-packages/mewpy/cobra/com/analysis.py:418: UserWarning: MRO: Failed to find a valid solution for: vpi5482
  warn('MRO: Failed to find a valid solution for: ' + org_id)


In [41]:
print(f'Community score: {score}\n')

print('Total competition for resources:\n')
print(MRO.community_medium)
print()
print('By individual:\n')

for ind in MRO.individual_media.keys():
    print(f'Strain:{ind}\t{", ".join(met for met in MRO.individual_media[ind])}')

Community score: None

Total competition for resources:



AttributeError: 'NoneType' object has no attribute 'community_medium'

In [ ]:
MRO.individual_media.vpi5482

In [ ]:
MRO.individual_media.atcc8492